# Make Vertex-Wise Connectivity Matrices in `.dconn` format with Midnight Scan Club Data

In [1]:
import neuropythy as ny
import numpy as np
import os
import json
import gc
import glob
import nibabel as nib

## Function for readaing in one Cifti file in `dtseries.nii` format and writing out the connectivity matrix as a `.dconn.nii` file

* Use nibabel to load in Cifti2
* Make correlation matrix with numpy from the vertex-wise timseries data
* Replace the data in the Cifti2 file with the correlation matrix, header based on previous header
* Write out new Cifti2 `.dconn.nii` file

In [2]:
def make_dconn_correlation_matrix_msc(filename):# The filename we are loading in:
    print(f'Preparing to make dconn file for {filename}')
    # Get the subject and load the CIFTI file
    cii = nib.cifti2.load(filename)
    bold = cii.dataobj # just get the whole matrix, don't split it
    
    
    print('Making correlation matrix...')
    # Calculate the correlation coefficients (takes 1-2 minutes)
    r = np.corrcoef(bold, rowvar=False, dtype=np.float32)
    del bold
    
    # To make a new CIFTI we need to change the CIFTI axes from time x vertices
    # to vertices x vertices:
    print('Making new cifti...')
    brain_axis = cii.header.get_axis(1)
    new_cii = nib.cifti2.Cifti2Image(r, header=(brain_axis, brain_axis), nifti_header=cii.nifti_header)
    
    del r
    del cii
    gc.collect()

   
    # Write out the filename
    print('Saving dconn...')
    outfile_ext = filename.replace('dtseries', 'dconn')
    new_cii.to_filename(outfile_ext)
    print('Done!')
    
    # to save memory
    del new_cii
    gc.collect()

## Get list of msc_files

In [3]:
msc_files = glob.glob('*dtseries.nii')
msc_files

['sub01_cifti2.dtseries.nii']

In [4]:
# test and make sure can be loaded as cifti!
#test_file = nib.cifti2.load(msc_files[0])

## Run function to make connectivity matrices

In [5]:
for i in msc_files:
    make_dconn_correlation_matrix_msc(msc_files[i])

vox offset (=5.21799e+06) not divisible by 16, not SPM compatible; leaving at current value


Preparing to make dconn file for sub01_cifti2.dtseries.nii


/home/ubuntu/miniconda3/envs/networks/lib/python3.8/site-packages/nibabel/nifti1.py:588: UserWarning: Extension size is not a multiple of 16 bytes; Assuming size is correct and hoping for the best
  warnings.warn(
vox offset (=5.21799e+06) not divisible by 16, not SPM compatible; leaving at current value


Making correlation matrix...
Making new cifti...
Saving dconn...
Done!
